# Reconstruction of the Invariant Mass of the Z boson in C++

In this analysis, we are focusing on identifying the decay of the [Z bosons](https://en.wikipedia.org/wiki/W_and_Z_bosons), a fundamental particle responsible for mediating the weak nuclear force. Let's get started!

<CENTER>
    <a href="http://opendata.atlas.cern" class="icons"><img src="../../images/ATLASOD.gif" style="width:40%"></a>
</CENTER>

## What is the Z Boson?
The Z boson is one of the mediators of the weak force, which is responsible for processes such as [beta decay](https://en.wikipedia.org/wiki/Beta_decay) in atomic nuclei. It interacts with all known fermions (quarks and leptons), but unlike the W boson, it does not change the type (flavor) of particle it interacts with. The Z boson couples to both [left-handed and right-handed](https://en.wikipedia.org/wiki/Chirality_(physics)) particles, making its behavior distinct from the charged W boson.

Since the Z boson is electrically neutral, its decay products must have balanced charges. The decays of the Z boson into leptons (electrons, muons, and taus) are particularly useful for experimental studies because these particles can be precisely measured in detectors, giving a clear signature of the Z boson's presence.

## The Decay of the Z Boson

The Z boson decays rapidly due to its high mass, with a mean lifetime of around 3 × 10$^{-25}$ seconds. Its decay channels include hadrons (quarks) and leptons, but in this analysis, we are particularly interested in the lepton channels because they produce clean final states that are easier to measure.

In this notebook, we are looking for events where the Z boson decays to:
- Two leptons of the same flavor but opposite charge (e.g., Z → e$^+$e$^-$ or Z → μ$^+$μ$^-$). Showed in the image on the left.
- In some cases, four leptons, which could result from intermediate states. Showed in the image on the right.

<table style="width:100%; text-align:center;">
  <tr>
    <td><img src="../../images/Z_ElectronPositron.png" style="width:65%"></td>
    <td><img src="../../images/fig01a.png" style="width:50%"></td>
  </tr>
</table>


The aim of this analysis is to plot the Z peak, which refers to the resonant mass of the Z boson observed in the invariant mass distribution of the two leptons. By plotting the invariant mass of the two-lepton system, we expect to see a peak around 91 GeV/c², which corresponds to the mass of the Z boson.

## Running a Jupyter notebook
A Jupyter notebook consists of cell blocks, each containing lines of Python code. Each cell can be run independently of each other, yielding respective outputs below the cells. Conventionally,cells are run in order from top to bottom.

- To run the whole notebook, in the top menu click Cell $\to$ Run All.
- To propagate a change you've made to a piece of code, click Cell $\to$ Run All Below.
- You can also run a single code cell, by clicking Cell $\to$ Run Cells, or using the keyboard shortcut Shift+Enter.

For more information, refer to [How To Use Jupyter Notebooks](https://www.codecademy.com/article/how-to-use-jupyter-notebooks).

By the end of this notebook you will be able to:
1. Learn to process large data sets using cuts
2. Understand some general principles of a particle physics analysis
3. Discover the Z boson!

## The analysis

The following section details the steps taken to analyze the Z boson decays from our data sample. The analysis revolves around reading, processing, and visualizing events where the Z boson decays into leptons.

The first step is to import the `ROOT` library, which is used for handling `.root` files, a widely used data format in High Energy Physics (HEP) for storing large datasets.

In [1]:
#include <iostream>
#include <string>
#include <stdio.h>
#include <time.h>

To visualize the histogram interactively, we enable the JSROOT magic function. This allows us to display the ROOT histograms directly in the notebook.

In [2]:
%jsroot on

Because we would like to use more than one ROOT input file, the best option is to use a TChain object. This allows to "chain" several samples into a single structure that we can later loop over

In [3]:
TChain *MonteCarlo = new TChain("mini");

Next, we open the data file, which contains simulated events from particle collisions. These events are stored in `.root` files. You can choose between different datasets:
- The **2-lepton sample**, where events involve two leptons, possibly from Z boson decays.
- The **4-lepton sample**, for more complex events involving four leptons.

In [4]:
MonteCarlo->Add("http://opendata.cern.ch//eos/opendata/atlas/OutreachDatasets/2016-07-29/MC/mc_147770.Zee.root"); // 8 TeV sample
MonteCarlo->Add("http://opendata.cern.ch//eos/opendata/atlas/OutreachDatasets/2016-07-29/MC/mc_147771.Zmumu.root"); // 8 TeV sample

We're extracting generic variables, like the RunNumber. This is a variable that heps to identify the sample inside the analysis code

In [5]:
Int_t MonteCarlo_runNumber = -1;   //Run identifier

Now we're going to create the leptons variables

In [6]:
UInt_t  MonteCarlo_lep_n = -1,      //number of preselected leptons
       MonteCarlo_lep_type[5];     //number signifying the lepton type (e, mu, tau) of the lepton

Float_t MonteCarlo_lep_pt[5],      //transverse momentum of the lepton
        MonteCarlo_lep_charge[5],  //charge of the lep
        MonteCarlo_lep_eta[5],     //pseudorapidity of the lepton
        MonteCarlo_lep_phi[5],     //azimuthal angle of the lepton
        MonteCarlo_lep_E[5];       //energy of the lepton

Next we're filling these variables with the content inside the input `ntuples`, or root files.

In [7]:
//MonteCarlo
MonteCarlo->SetBranchAddress("channelNumber", &MonteCarlo_runNumber);

MonteCarlo->SetBranchAddress("lep_pt",        &MonteCarlo_lep_pt);
MonteCarlo->SetBranchAddress("lep_charge",    &MonteCarlo_lep_charge);
MonteCarlo->SetBranchAddress("lep_n",         &MonteCarlo_lep_n);
MonteCarlo->SetBranchAddress("lep_type",      &MonteCarlo_lep_type);
MonteCarlo->SetBranchAddress("lep_eta",       &MonteCarlo_lep_eta);
MonteCarlo->SetBranchAddress("lep_phi",       &MonteCarlo_lep_phi);
MonteCarlo->SetBranchAddress("lep_E",         &MonteCarlo_lep_E);

We're creating two (2) histograms for this example. The plan is to fill them with events coming from different input MC samples

The histograms are created to store the invariant mass distribution of the Z boson decay products. One histogram for the decay into an electron-positron pair, and another for the decay into a muon-antimuon pair. These histograms are binned into 20 intervals, ranging from 0 GeV to 150 GeV on the x-axis (mass), while the y-axis represents the number of events. This range is centered around the Z boson mass (~91 GeV).

In [8]:
//Invariant mass histograms definition
TH1F *h_M_Zee   = new TH1F("h_M_Zee"  ,  "Invariant Mass of the Z (Zee) ; M_Zee[GeV] ; # de eventos" ,20,0,150);
TH1F *h_M_Zmumu = new TH1F("h_M_Zmumu","Invariant Mass of the Z (Zmumu) ; M_Zmumu[GeV] ; # de eventos" ,20,0,150);

To fill the histogram, we loop over each event in the TTree, applying several cuts to select the events of interest. The goal is to isolate Z boson decays into two leptons:

- **Cut #1: Exactly two "good" leptons**. This ensures that the event contains two highly energetic leptons, which are necessary for reconstructing the Z boson mass.
- **Cut #2: Opposite charge leptons**. Since the Z boson is neutral, it decays into a lepton and its antiparticle (opposite charge).
- **Cut #3: Same lepton family**. We select events where both leptons are either electrons (Z → e$^+$e$^-$) or muons (Z → μ$^+$μ$^-$).

For each event passing the cuts, we calculate the invariant mass using the four-momenta of the leptons and fill the histogram.

In [9]:
int nentriesMC, nbytesMC, i;
nentriesMC = (Int_t)MonteCarlo->GetEntries();

std::cout << " Total number of entries to analyze: " << nentriesMC << std::endl;

for (i=0;i< nentriesMC; i++)
{
    nbytesMC =  MonteCarlo->GetEntry(i);
           
    //Cut #1: Exactly two good leptons with pT > 25GeV
    if(MonteCarlo_lep_n ==2 || MonteCarlo_lep_pt[2] <25000.)
    {
      // TLorentzVector definitions 
        TLorentzVector Lepton_1  = TLorentzVector();
        TLorentzVector Lepton_2  = TLorentzVector();
        
        Lepton_1.SetPtEtaPhiE(MonteCarlo_lep_pt[0], MonteCarlo_lep_eta[0], MonteCarlo_lep_phi[0],MonteCarlo_lep_E[0]);
        Lepton_2.SetPtEtaPhiE(MonteCarlo_lep_pt[1], MonteCarlo_lep_eta[1], MonteCarlo_lep_phi[1],MonteCarlo_lep_E[1]);
      //Cut #2: The two selected leptons must have opposite charge
      int OS = MonteCarlo_lep_charge[0]*MonteCarlo_lep_charge[1];
      if(OS == -1)
      {  
           //Cut #3: The two selected leptons have the same flauvour
           if(MonteCarlo_lep_type[0] == MonteCarlo_lep_type[1])
           {
               //Calculation of the Invariant Mass using TLorentz vectors
               TLorentzVector Lepton_12 = Lepton_1 + Lepton_2;
               
               float mass_inv_GeV = Lepton_12.M()/1000.;

                    /*The runNumber value helps to separate the samples*/
                    if(MonteCarlo_runNumber ==147770) // where 147770 == Zee
                    {
                        h_M_Zee->Fill(mass_inv_GeV); // invariant mass of two electrons
                    }
                    else if(MonteCarlo_runNumber ==147771) // where 147771 == Zmumu
                    {
                        h_M_Zmumu->Fill(mass_inv_GeV); // invariant mass of two muons 
                    }
                
            }
        }
    }
}

 Total number of entries to analyze: 15000000


Below, we select some parameters for the look of the histograms.

In [10]:
h_M_Zee->SetMarkerSize(2.0);
h_M_Zee->SetLineColor(kBlue);
h_M_Zee->SetFillColor(kBlue-10);

h_M_Zmumu->SetMarkerSize(2.0);
h_M_Zmumu->SetLineColor(kGreen);
h_M_Zmumu->SetFillColor(kGreen-10);

In [ ]:
Lastly, we plot the invariant mass from both samples.

In [11]:
TCanvas *cz = new TCanvas("cz","cz",10,10,900,600);
    TText tz; tz.SetTextFont(42); tz.SetTextAlign(21);
    cz->Divide(2,1);
    cz->cd(1); h_M_Zee->Draw();
    cz->cd(2); h_M_Zmumu->Draw();

cz->Draw();

The two histograms above show the invariant mass distributions for Z boson decays into two electrons (Z $\rightarrow$ ee) and two muons (Z $\rightarrow$ μμ). These distributions are used to identify the Z boson mass peak, expected around 91 GeV, as per the Standard Model.

- **Left Plot (Z $\rightarrow$ ee)**: The invariant mass distribution of electron pairs peaks at 84.14 GeV with a standard deviation of 20.28 GeV. While the peak appears lower than the nominal Z boson mass, it falls within the expected range when taking into account the standard deviation. This is consistent with the simplicity of the analysis, which involves only a few cuts and limited event selection criteria.
- **Right Plot (Z $\rightarrow$ μμ)**: The invariant mass distribution of muon pairs peaks at 86.57 GeV with a standard deviation of 17.17 GeV. This result is similarly in line with the expected Z boson mass, considering the standard deviation. Again, the relatively basic analysis with few cuts and limited optimization means the result falls well within the acceptable range, given the inherent uncertainties.

In both decay channels, the Z boson mass peak is clearly visible, and the differences in the peak positions reflect the simplicity of the analysis and the natural experimental resolution. With more refined cuts and additional optimization, the results can be further aligned with the exact Z boson mass of 91 GeV.